# Learn to Rank

## Import 

In [2]:
import pandas as pd
import json
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import pickle
import math 

In [3]:
df = pd.read_pickle('/home/adam/Steph_C/my_thesis/data/FE_by_postoal_code_without_review.pkl')

## DF Construction
* add relevance

In [4]:
df.columns

Index(['business_id', 'stars_x', 'useful', 'funny', 'cool', 'text', 'date',
       'name', 'address', 'city', 'state', 'postal_code', 'stars_y',
       'review_count', 'is_open', 'attributes', 'categories', 'hours',
       'density', 'entropy', 'competitiveness'],
      dtype='object')

In [5]:
df = df.drop(columns=[ 'business_id', 'stars_x', 'useful', 'funny', 'cool', 'text',
       'date','address','is_open', 'stars_y','attributes','hours'])

In [6]:
# sort by name and review count 
df = df.sort_values(['name', 'review_count'],
              ascending = [True, False]).reset_index(drop=True)

In [7]:
df

,name,city,state,postal_code,review_count,categories,density,entropy,competitiveness
0,1933 Lounge,Indianapolis,IN,46225,160,"Bars, Nightlife, Restaurants, American (Tradit...",6,1.791759,-0.166667
1,1933 Lounge,Fishers,IN,46037,150,"Steakhouses, Nightlife, Bars, Wine Bars, Resta...",6,1.791759,-0.166667
2,4 Rivers Smokehouse,Tampa,FL,33618,572,"Barbeque, Smokehouse, Restaurants, Food, South...",13,2.564949,-0.076923
3,4 Rivers Smokehouse,Tampa Bay,FL,33609,343,"Barbeque, Smokehouse, Food, Event Planning & S...",11,2.397895,-0.090909
4,54th Street Grill & Bar,St. Louis County,MO,63126,146,"American (Traditional), Bars, Nightlife, Burge...",3,1.098612,-0.333333
...,...,...,...,...,...,...,...,...,...
1456,Zydeco's,Belle Chasse,LA,70037,161,"Restaurants, Cajun/Creole, American (New), Sea...",1,-0.0,-1.0
1457,sweetgreen,Philadelphia,PA,19103,213,"Vegan, Soup, Vegetarian, Salad, Restaurants",19,2.944439,-0.052632
1458,sweetgreen,King Of Prussia,PA,19406,91,"Restaurants, Salad, Vegetarian",19,2.944439,-0.052632
1459,uBreakiFix,New Orleans,LA,70130,105,"Local Services, Electronics Repair, IT Service...",18,3.211524,-0.055556


In [9]:
len(Counter(df.name))

566

In [10]:
# create relevance score
cnt = 0
for i in Counter(df.name):
    tmp = df[df.name==i].reset_index(drop=True)
    tmp['relevance']=''
    score = 6
    for j in range(len(tmp)):
        tmp['relevance'][j]=score
        score -=1
    if cnt ==0 :
        new_df = tmp
        cnt+=1
    new_df = pd.concat([new_df,tmp])

/home/adam/.local/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
for i in Counter(new_df.name):
    tmp = new_df[new_df.name==i]
    print(f'Restaurant : {i} , cities : {Counter(tmp.postal_code)}')

Restaurant : 1933 Lounge , cities : Counter({'46225': 2, '46037': 2})
Restaurant : 4 Rivers Smokehouse , cities : Counter({'33618': 1, '33609': 1})
Restaurant : 54th Street Grill & Bar , cities : Counter({'63126': 1, '62269': 1})
Restaurant : 55 South , cities : Counter({'37064': 1, '37027': 1})
Restaurant : 7venth Sun Brewery , cities : Counter({'34698': 1, '33604': 1})
Restaurant : A La Mousse , cities : Counter({'19107': 1, '19146': 1})
Restaurant : Acme Oyster House , cities : Counter({'70130': 1, '70002': 1})
Restaurant : Acropolis Greek Taverna , cities : Counter({'33605': 1, '33701': 1, '33613': 1, '33578': 1, '33609': 1})
Restaurant : Alamo Rent A Car , cities : Counter({'33607': 1, '37214': 1})
Restaurant : Ale Emporium , cities : Counter({'46250': 1, '46143': 1})
Restaurant : All My Sons Moving & Storage , cities : Counter({'46038': 1, '37207': 1, '08002': 1, '63074': 1})
Restaurant : Amazing Lash Studio , cities : Counter({'85704': 1, '46268': 1, '19087': 1})
Restaurant : Am

In [8]:
# # breakdown affinity & complementary
# from operator import itemgetter
# indices = range(len(df['affinity'][0]))
# a_df = df['affinity'].transform({f'affinity_{i+1}': itemgetter(i) for i in indices})

# indices = range(len(df['complementary'][0]))
# c_df = df['complementary'].transform({f'complementary_{i+1}': itemgetter(i) for i in indices})

In [9]:
# new_df = new_df.join(a_df)
# new_df = new_df.join(c_df)

## Model

### Create Dataset

In [13]:
new_df.columns

Index(['name', 'city', 'state', 'postal_code', 'review_count', 'categories',
       'density', 'entropy', 'competitiveness', 'relevance'],
      dtype='object')

In [14]:
all_postal_code = [i for i in Counter(new_df.postal_code)]

AttributeError: 'Series' object has no attribute 'info'

In [24]:
# split train test 
cnt = 0

for i in Counter(df.name):
    tmp = new_df[new_df.name==i]
    
    if cnt ==0:
        
        train_df = tmp.iloc[:-1,:]
        test_df = tmp.iloc[-1:,:]
        cnt+=1
    else:
        train_df = pd.concat([train_df ,tmp.iloc[:-1,:] ])
        test_df = pd.concat([test_df ,tmp.iloc[-1:,:] ])
    


In [26]:
print(train_df.shape , test_df.shape)

(897, 10) (566, 10)


In [35]:
features = ['name', 'city', 'state', 'postal_code', 'review_count', 'categories',
       'density', 'entropy', 'competitiveness',
       'affinity_1', 'affinity_2', 'affinity_3', 'affinity_4',
       'affinity_5', 'affinity_6', 'affinity_7', 'affinity_8', 'affinity_9',
       'affinity_10', 'complementary_1', 'complementary_2', 'complementary_3',
       'complementary_4', 'complementary_5', 'complementary_6',
       'complementary_7', 'complementary_8', 'complementary_9',
       'complementary_10']
target = 'relevance'

for i in ['density', 'entropy', 'competitiveness','postal_code'
       'relevance']:
       train_df[i] = train_df[i].astype('float')

### Training

In [30]:
train_df.columns

Index(['name', 'city', 'state', 'postal_code', 'review_count', 'categories',
       'density', 'entropy', 'competitiveness', 'relevance'],
      dtype='object')

In [28]:
print(len(Counter(train_df.postal_code)), len(Counter(test_df.postal_code)))

263 251


In [43]:
from lightgbm import LGBMRanker
train_features = ['density', 'entropy', 'competitiveness']

model = LGBMRanker(objective="lambdarank")
model.fit(train_df[train_features],train_df[['relevance']],group=[i for i in Counter(train_df.postal_code).values()],eval_set=[(test_df[train_features],test_df[['relevance']])],eval_group=[i for i in Counter(test_df.postal_code).values()],eval_metric=['ndcg'])

ValueError: Length of eval_group should be equal to eval_set

In [45]:
len([(test_df[train_features],test_df['relevance'])])

1

In [46]:
[(test_df[train_features],test_df['relevance'])]

[(   density   entropy competitiveness
  1        6  1.791759       -0.166667
  1       11  2.397895       -0.090909
  1        4  1.386294           -0.25
  1       10  2.302585            -0.1
  1        3  1.098612       -0.333333
  ..     ...       ...             ...
  1        4  1.386294           -0.25
  4        4  1.386294           -0.25
  1        1      -0.0            -1.0
  1       19  2.944439       -0.052632
  1       13  2.564949       -0.076923
  
  [566 rows x 3 columns],
  1    5
  1    5
  1    5
  1    5
  1    5
      ..
  1    5
  4    2
  1    5
  1    5
  1    5
  Name: relevance, Length: 566, dtype: object)]